In [ ]:
import numpy as np
from datetime import datetime
from datetime import timedelta
import pandas as pd
import matplotlib.pyplot as plt
import re
import timeit
import datetime
import xgboost as xgb
import math
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import plotly.graph_objs as go
import folium
import json
import utm
import scipy
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import joblib
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
import folium
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import datetime
import scipy.stats as stats
import plotly.express as px
pd.set_option('mode.chained_assignment', None)

# libraries for import all file names
from prettytable import PrettyTable
from os import listdir
from os.path import isfile, join
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import accuracy_score , mean_squared_error
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from yellowbrick.model_selection import FeatureImportances
from sklearn.linear_model import LinearRegression


## Analisis Anomalies

In [ ]:
def calcularAnomaly(df, col, cont=0.005):
    #Crear una versio horaria del dataset
    df['timestamp'] = pd.to_datetime(df.index)
    df = df.set_index('timestamp').resample("H").mean().reset_index()
    
    #Usem el metode de Isolation Forest per trobar anomalies a les precipitacions
    output = df[[col, 'timestamp']]
    temp = df[[col, 'timestamp']]
    model=IsolationForest(n_estimators=50, max_samples='auto', contamination=cont,max_features=1.0)
    model.fit(temp[[col]].values)
    output['outliers']=pd.Series(model.predict(temp[[col]].values)).apply(lambda x: 'yes' if (x == -1) else 'no' )
    output.rename(columns={col:'data'}, inplace=True)
    
    return output

In [ ]:
# Carregar dades pluja
def loadDataPrecipitacions():
    precipitacions = pd.read_excel("finalsDF/DF_SMC.xlsx", index_col=0)
    return precipitacions

In [ ]:
# Carregar dades Alt Ter
def loadDataAltTer():
    altTer = pd.read_excel("Dataframes/df_imputedAltTerKNN.xlsx", index_col=0)
    santJoan = calcularAnomaly(altTer, "L17167-72-00001")
    massiesRoda = calcularAnomaly(altTer, "L08116-72-00002")
    
    return santJoan, massiesRoda

In [ ]:
# Carregar dades Baix Ter
def loadDataBaixTer():
    baixTer = pd.read_excel("Dataframes/df_imputedBaixTerKNN.xlsx", index_col=0)
    pasteralCabal = calcularAnomaly(baixTer, "F001242")
    colomers = calcularAnomaly(baixTer, "L17055-72-00002")
    torroellaMontegri = calcularAnomaly(baixTer, "L17199-72-00001")
    
    return pasteralCabal, colomers, torroellaMontegri

In [ ]:
def selectTimespan(df, start_date, end_date):
    df = df.loc[(df['timestamp'] >= start_date) &  (df['timestamp'] <= end_date)]
    return df

In [ ]:
# Carregar dades rius

alt1, alt2 = loadDataAltTer()
baix1, baix2, baix3 = loadDataBaixTer()


## Periode Temporal

### Cerca de Periodes Temporals amb Anomalies (Alt Ter)

In [ ]:
fig= px.scatter(alt1, x='timestamp', y='data', color='outliers',title='Anomalies Sant Joan')
fig.update_xaxes(
    rangeslider_visible=True,
)
fig2 = px.scatter(alt2, x='timestamp', y='data', color='outliers',title='Anomalies Massies Roda')
fig2.update_xaxes(
    rangeslider_visible=True,
)

fig.show()
fig2.show()

Trobem anomalies a les dues estacions del Alt Ter a les següents dates:
- 2010    
    - Maig 2010 (42-165 l/m3)
    - Octubre 2010 (132 - 335 l/m3)
- 2011
    - Març 2011 (161 - 541 l/m3) (https://www.elpuntavui.cat/territori/article/11-mediambient/384345-el-riu-ter-sendu-un-tram-del-carril-bici-a-salt-i-un-passallis-a-bescano.html)
    - Novembre 2011 (44 - 208 l/m3)
- 2013
    - Març 2013 (137 - 361 l/m3) (https://www.garrotxadigital.cat/2013/03/06/el-riu-ter-multiplica-per-cent-el-seu-cabal/)
    - Novembre 2013 (43 - 174 l/m3) (https://www.elpuntavui.cat/societat/article/5-societat/694602-proteccio-civil-alerta-de-laugment-del-cabal-dels-rius-besos-ter-i-fluvia.html)
- 2014
    - Novembre 2014 (204 - 1102 l/m3) (https://www.elpuntavui.cat/societat/article/5-societat/800291-situacio-dralerta-al-ter.html)
- 2015
    - Novembre 2015 (162 - 718 l/m3) (https://www.ripollesdigital.cat/noticia/65872/el-cabal-del-riu-ter-al-seu-pas-per-ripoll-sacosta-als-400-metres-cubics-per-segon-aquesta)
- 2017
    - Febrer 2017 (76 - 219 l/m3)
- 2018
    - Abril 2018 (95 - 341 l/m3) (https://www.elter.net/noticia/22295/les-pluges-multipliquen-21-cabal-del-riu-ter-osona)
    - Octubre 2018 (359 - 1573 l/m3) (https://www.naciodigital.cat/noticia/164869/fotos-videos-ter-es-desborda-al-seu-pas-osona) (Huracà Leslie)
- 2019
    - Octubre 2019 (108 - 315 l/m3) (https://www.elpuntavui.cat/societat/article/5-societat/1684802-el-riu-ter-baixa-molt-ple-al-seu-pas-per-roda-de-ter.html)
- 2020
    - Gener 2020 (162 - 1183 l/m3) (https://www.consorcidelter.cat/actualitat/noticies/2020/01/27/gloria-al-ter) (Temporal Glòria)
    - Juny 2020 (44 - 242 l/m3)
    - Agost 2020 (44 - 315 l/m3)
    - November 2020 (57 - 250 l/m3) (https://www.ccma.cat/324/activats-el-protocols-per-confinar-els-veins-que-viuen-prop-de-la-llera-del-riu-ter/noticia/2983502/)
    

### Cerca de Periodes Temporals amb Anomalies (Baix Ter)

In [ ]:
fig= px.scatter(baix1, x='timestamp', y='data', color='outliers',title='Anomalies Pasteral - Cabal')
fig.update_xaxes(
    rangeslider_visible=True,
)
fig2 = px.scatter(baix2, x='timestamp', y='data', color='outliers',title='Anomalies Colomers')
fig2.update_xaxes(
    rangeslider_visible=True,
)
fig3 = px.scatter(baix3, x='timestamp', y='data', color='outliers',title='Anomalies Torroella - Montegrí')
fig3.update_xaxes(
    rangeslider_visible=True,
)

fig.show()
fig2.show()
fig3.show()

Trobem anomalies a les dues estacions del Alt Ter a les següents dates:
- 2011
    - Març 2011 (357 - 544 - 245 l/m3) (També anomalia a part alta)
    - Novembre 2011 (209 - 684 - 395 l/m3) (També anomalia a part alta)
- 2013
    - Març 2013 (681 - 342 l/m3) (NOMËS A COLOMERS I TORROELLA - MONTEGRÍ) (També anomalía a part alta)
- 2014
    - Decembre 2014 (286 - 1131 - 418 l/m3) (http://www.elter.net/noticia/17337/societat/manlleu/forta-crescuda-del-riu-ter-arrossega-vehicle-malars-sense-ferits) (Coincideix amb anomalia a la part alta del riu a finals de Novembre)
- 2018
    - Octubre 2018 (248 - 523 - 269 l/m3) (Coincideix amb anomalia a la part alta)
    - Novembre 2018 (821 - 490 l/m3) (NOMËS A COLOMERS I TORROELLA - MONTEGRÍ) (Coincideix amb anomalia a la part alta del riu a finals de Octubre)
- 2020
    - Gener 2020 (1202 - 1147 - 863 l/m3) (Temporal Gloria, coincideix amb anomalia a la part alta del riu)
    - Abril 2020 (763 - 387 l/m3) (NOMES A COLOMERS I TOROELLA - MONTEGRÍ) (Petita pujada a Massies Roda)

### Analisis Periode Especific (Temporal Gloria)

In [ ]:
#Seleccionem el periode temporal (en aquesta part el temporal Gloria)

start_date = datetime.datetime.strptime("20/01/2020", "%d/%m/%Y")
end_date = datetime.datetime.strptime("26/01/2020", "%d/%m/%Y")

alt1 = selectTimespan(alt1, start_date, end_date)
alt2 = selectTimespan(alt2, start_date, end_date)
baix1 = selectTimespan(baix1, start_date, end_date)
baix2 = selectTimespan(baix2, start_date, end_date)
baix3 = selectTimespan(baix3, start_date, end_date)

## Pearson

Ordre rius

- Alt Ter: Sant Joan de les Abadesses (L17167-72-00001) > Massies de Roda (L08116-72-00002)

- Baix Ter: Pasteral-Cabal (F001242) > Colomers (L17055-72-00002) > Torroella de Montgrí (L17199-72-00001)

In [ ]:
def pearson_forward_lag(x, y, debug=False, n=40,m=40, p_ini=0):
    timeList = []
    rList = []
    pList = []
    offset = 0
    time = "00:00"
    timeStr = "%H:%M"
    time = datetime.datetime.strptime(time, timeStr)
    while offset < m:
        r, p = stats.pearsonr(x[p_ini:n], y[p_ini+offset:offset+n])
        if debug == True:
            print(f"Time Lag: +{time.strftime('%H:%M')}h , Pearson r: {r} and p-value: {p}")
        timeList.append(time)
        rList.append(r)
        pList.append(p)
        time = time + timedelta(minutes=30)
        offset += 1
        
    return timeList, rList, pList

In [ ]:
def normalize_column(df, col):
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    return df

## Analisis Alt Ter (Sant Joan de les Abadesses > Massies de Roda)

In [ ]:
# Analisis Alt Ter
t, r, p = pearson_forward_lag(alt1['data'], alt2['data'])

In [ ]:
# Fer un dataframe amb les dades aconseguides
d = {'r':r, 'p':p}
df = pd.DataFrame(data=d,index=t)
df.index = df.index.strftime('%H:%M')
fig = px.line(df, x=df.index, y=["r", "p"], markers=True)
fig

## Analis Baix Ter (Pasteral-Cabal > Colomers)

In [ ]:
# Analisis Alt Ter
t, r, p = pearson_forward_lag(baix1['data'], baix2['data'])


In [ ]:
# Fer un dataframe amb les dades aconseguides
d = {'r':r, 'p':p}
df = pd.DataFrame(data=d,index=t)
df.index = df.index.strftime('%H:%M')
fig = px.line(df, x=df.index, y=["r", "p"], markers=True)
fig

## Analis Baix Ter (Colomers > Torroella de Montegri)

In [ ]:
# Analisis Alt Ter
t, r, p = pearson_forward_lag(baix2['data'], baix3['data'])

In [ ]:
# Fer un dataframe amb les dades aconseguides
d = {'r':r, 'p':p}
df = pd.DataFrame(data=d,index=t)
df.index = df.index.strftime('%H:%M')
fig = px.line(df, x=df.index, y=["r", "p"], markers=True)
fig